In [1]:
import pandas as pd
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By
import time
import json
from selenium.webdriver.chrome.service import Service
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")

### GET informations about each 35 destinations

In [2]:
list_35_destinations = [
"Mont Saint Michel",
"St Malo",
"Bayeux",
"Le Havre",
"Rouen",
"Paris",
"Amiens",
"Lille",
"Strasbourg",
"Chateau du Haut Koenigsbourg",
"Colmar",
"Eguisheim",
"Besancon",
"Dijon",
"Annecy",
"Grenoble",
"Lyon",
"Gorges du Verdon",
"Bormes les Mimosas",
"Cassis",
"Marseille",
"Aix en Provence",
"Avignon",
"Uzes",
"Nimes",
"Aigues Mortes",
"Saintes Maries de la mer",
"Collioure",
"Carcassonne",
"Ariege",
"Toulouse",
"Montauban",
"Biarritz",
"Bayonne",
"La Rochelle"
]


In [3]:

for city in list_35_destinations:

    time.sleep(30)
    path=Service("C:\\Program Files (x86)\\Google\\Chrome\\Application\\chromedriver\\chromedriver.exe")
    browser = webdriver.Chrome(chrome_options=options, service=path)
    browser.get("https://www.booking.com/")
    browser.maximize_window()
    time.sleep(5)

    try:
        rgpd_button = browser.find_element(By.XPATH, '//*[@id="onetrust-reject-all-handler"]')
        rgpd_button.click()
    except:
        rgpd_button = browser.find_element(By.XPATH, '//*[@id="onetrust-accept-btn-handler"]')
        rgpd_button.click()
    time.sleep(3)

    text_box = browser.find_element(By.NAME,'ss')
    submit_button = browser.find_element(By.XPATH,'//*[@id="frm"]/div[1]/div[4]/div[2]/button')

    text_box.send_keys(city)
    submit_button.click()
    time.sleep(8)

    ##### SORT RESULT WITH STARS DESCENDING ORDER ####

    # Be Carefull with this piece of code, it works but XPATH of button change regularly
    # and needs to be updated manually

    # sort_result_button = WebDriverWait(browser, timeout=20).until(lambda d: d.find_element(By.XPATH, '//*[@id="search_results_table"]/div[2]/div/div/div/div[2]/span/button'))
    # #sort_result_button = browser.find_element(By.XPATH, '//*[@id="search_results_table"]/div[2]/div/div/div/div[2]/span/button')
    # sort_result_button.click()                          
    # time.sleep(3)

    # try:
    #     button_step_1 = browser.find_element(By.XPATH, '/html/body/div[18]/div/div/div/div/ul/li[3]')
    #     button_step_2 = button_step_1.find_element(By.TAG_NAME, 'button').click()                      

    # except NoSuchElementException:
    #     etoiles_ordre_decroissant_button = browser.find_element(By.XPATH, '//*[@id="__bui-c3-1"]/div/div/div/div/ul/li[3]/button') 
    #     etoiles_ordre_decroissant_button.click()
          
    # time.sleep(5)
    #################################################


    #### INIT LOOP TO SCRAP 25 RESULTS FOR THIS CITY NAME ####
    list_results = []
    count=1


    property_cards = browser.find_elements(By.CSS_SELECTOR, 'div[data-testid="property-card"]')

    for property in property_cards:
        new_result = {}

        print(count)

        try:
            name = property.find_element(By.CSS_SELECTOR,'div[data-testid="title"]').text
        except:
            name = 'N/A'
        print(name)

        try:
            link = property.find_element(By.CSS_SELECTOR,'a[data-testid="title-link"]').get_attribute('href')
        except:
            link = 'N/A'
        print(link)

        try:
            review_score, _, review_count = property.find_element(By.CSS_SELECTOR, '[data-testid="review-score"]').text.split('\n')
        except:
            review_score = 'N/A' 
            review_count = 'N/A'
        print(review_score, review_count)

        try:
            area = property.find_element(By.CSS_SELECTOR, '[data-testid="address"]').text
        except:
            area = 'N/A'
        print(area)
        

        ##### RECORDING RESULTS AND OPEN A NEW WINDOW TO SCRAP MORE INFORMATIONS ########    

        new_result['name'] = name
        new_result['link'] = link
        new_result['review_score'] = review_score
        new_result['review_count'] = review_count
        #new_result['price'] = price
        new_result['area'] = area


        ##################

        # init browser2
        time.sleep(2)
        browser2 = webdriver.Chrome(chrome_options=options, service=path)
        browser2.get(link)
        browser2.maximize_window()
        time.sleep(10)

        #RGPD Button
        try:
            time.sleep(3)
            rgpd_button = browser2.find_element(By.XPATH, '//*[@id="onetrust-reject-all-handler"]')
            rgpd_button.click()
        except:
            time.sleep(3)
            rgpd_button = browser2.find_element(By.XPATH, '//*[@id="onetrust-accept-btn-handler"]')
            rgpd_button.click()
        time.sleep(3)

        try:
            address = browser2.find_element(By.XPATH, '//*[@id="showMap2"]/span[1]').text
        except:
            address = 'N/A'

        new_result['address'] = address
        print(address)
        time.sleep(1)

        try:
            coordinates_gps = browser2.find_element(By.XPATH, '//*[@id="hotel_header"]').get_attribute('data-atlas-latlng')
        except:
            coordinates_gps = 'N/A'
            
        new_result['hotel_latitude'] = coordinates_gps.split(',')[0]
        new_result['hotel_longitude'] = coordinates_gps.split(',')[1]
        print(coordinates_gps)
        print()
        time.sleep(1)

        try:
            button_see_more = browser2.find_element(By.XPATH, '//*[@id="basiclayout"]/div[1]/div[2]/div/div/div[1]/div[1]/div[1]/div[2]/a').click()
            time.sleep(2)
        except:
            button_see_more = 'N/A'
        
        try:
            description = browser2.find_element(By.XPATH, '//*[@id="basiclayout"]/div[1]/div[2]/div/div/div[1]/div[1]/div[1]').text
        except:
            description = 'N/A'

        #### RECORDING ADDITIONNAL RESULTS AND CLOSING BROWSER 2 ####

        new_result['description'] = description
        time.sleep(1)

        # close browser 2
        browser2.close()

        ##################


        # add all informations about 25 hotels in a list
        list_results.append(new_result)

        count +=1
        time.sleep(3)
        ### END OF RESULTS PER CITY LOOP ###




    # create a Dataframe for this CITY and export as a csv file
    df = pd.DataFrame(list_results)
    timestr = time.strftime("%Y%m%d-%H%M%S")
    df.to_csv(f'save_output/{timestr}-{city}.csv', index=False)

    # create  and record a json file containing this list of dict
    # Serializing json
    json_object = json.dumps(list_results)
    # Writing to sample.json
    with open(f"save_output/{timestr}-{city}.json", "w") as outfile:
        outfile.write(json_object)
    
    time.sleep(15)
    browser.close()
    # END OF CITY LOOP



C:\Users\Christophe\AppData\Local\Temp\ipykernel_13340\2872148576.py:5: DeprecationWarning: use options instead of chrome_options
  browser = webdriver.Chrome(chrome_options=options, service=path)


1
Hôtel Vert
https://www.booking.com/hotel/fr/vert.fr.html?label=gen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24ARfIAQzYAQHoAQH4AQuIAgGoAgO4ArL80J0GwAIB0gIkN2EwYWFhMGMtNGIwYy00MzAxLTgyNDctM2RkOTU2ZDYzZjc12AIG4AIB&aid=304142&ucfs=1&arphpl=1&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=1&hapos=1&sr_order=popularity&srpvid=1da1671fedec03bb&srepoch=1672756799&from=searchresults#hotelTmpl
8,0 4 389 expériences vécues
Le Mont-Saint-Michel


C:\Users\Christophe\AppData\Local\Temp\ipykernel_13340\2872148576.py:99: DeprecationWarning: use options instead of chrome_options
  browser2 = webdriver.Chrome(chrome_options=options, service=path)


La Caserne, 50170 Le Mont-Saint-Michel, France
48.61470049,-1.50961697

2
Mercure Mont Saint Michel
https://www.booking.com/hotel/fr/mont-saint-michel.fr.html?label=gen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24ARfIAQzYAQHoAQH4AQuIAgGoAgO4ArL80J0GwAIB0gIkN2EwYWFhMGMtNGIwYy00MzAxLTgyNDctM2RkOTU2ZDYzZjc12AIG4AIB&aid=304142&ucfs=1&arphpl=1&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=2&hapos=2&sr_order=popularity&srpvid=1da1671fedec03bb&srepoch=1672756799&from_sustainable_property_sr=1&from=searchresults#hotelTmpl
8,2 3 570 expériences vécues
Le Mont-Saint-Michel
La Caserne, 50170 Le Mont-Saint-Michel, France
48.61424653,-1.51054502

3
La Mère Poulard
https://www.booking.com/hotel/fr/la-mere-poulard.fr.html?label=gen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24ARfIAQzYAQHoAQH4AQuIAgGoAgO4ArL80J0GwAIB0gIkN2EwYWFhMGMtNGIwYy00MzAxLTgyNDctM2RkOTU2ZDYzZjc12AIG4AIB&aid=304142&ucfs=1&arphpl=1&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=3&hapos=3&sr_

C:\Users\Christophe\AppData\Local\Temp\ipykernel_13340\2872148576.py:5: DeprecationWarning: use options instead of chrome_options
  browser = webdriver.Chrome(chrome_options=options, service=path)


...

C:\Users\Christophe\AppData\Local\Temp\ipykernel_13340\2872148576.py:5: DeprecationWarning: use options instead of chrome_options
  browser = webdriver.Chrome(chrome_options=options, service=path)


1
Le petit nid
https://www.booking.com/hotel/fr/le-petit-nid-montgaillard.fr.html?label=gen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24ARfIAQzYAQHoAQH4AQuIAgGoAgO4ArvG0p0GwAIB0gIkNGY3ZDA4YjYtNjY4OS00YTU0LWE5NDMtMDlkOGE0YTg3NGUx2AIG4AIB&aid=304142&ucfs=1&arphpl=1&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=1&hapos=1&sr_order=popularity&srpvid=4fb599a3b73b0183&srepoch=1672782664&from_sustainable_property_sr=1&from=searchresults#hotelTmpl
9,1 20 expériences vécues
Montgaillard


C:\Users\Christophe\AppData\Local\Temp\ipykernel_13340\2872148576.py:99: DeprecationWarning: use options instead of chrome_options
  browser2 = webdriver.Chrome(chrome_options=options, service=path)


RDC 1 Rue de la Place, 09330 Montgaillard, France
42.93227802,1.63516787

2
dtente sky lodge
https://www.booking.com/hotel/fr/dtente-sky-lodge.fr.html?label=gen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24ARfIAQzYAQHoAQH4AQuIAgGoAgO4ArvG0p0GwAIB0gIkNGY3ZDA4YjYtNjY4OS00YTU0LWE5NDMtMDlkOGE0YTg3NGUx2AIG4AIB&aid=304142&ucfs=1&arphpl=1&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=2&hapos=2&sr_order=popularity&srpvid=4fb599a3b73b0183&srepoch=1672782664&from_sustainable_property_sr=1&from=searchresults#hotelTmpl
9,4 94 expériences vécues
Massat
La Prado, 09320 Massat, France
42.89450398,1.32823379

3
Hostellerie de la Poste
https://www.booking.com/hotel/fr/hostellerie-de-la-poste-tarascon-sur-ariege.fr.html?label=gen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24ARfIAQzYAQHoAQH4AQuIAgGoAgO4ArvG0p0GwAIB0gIkNGY3ZDA4YjYtNjY4OS00YTU0LWE5NDMtMDlkOGE0YTg3NGUx2AIG4AIB&aid=304142&ucfs=1&arphpl=1&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=3&hapos=3&sr_order=

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="onetrust-accept-btn-handler"]"}
  (Session info: chrome=108.0.5359.125)
Stacktrace:
Backtrace:
	Ordinal0 [0x005EACD3+2075859]
	Ordinal0 [0x0057EE61+1633889]
	Ordinal0 [0x0047B7BD+571325]
	Ordinal0 [0x004AAC2F+764975]
	Ordinal0 [0x004AAE1B+765467]
	Ordinal0 [0x004DD0F2+970994]
	Ordinal0 [0x004C7364+881508]
	Ordinal0 [0x004DB56A+963946]
	Ordinal0 [0x004C7136+880950]
	Ordinal0 [0x0049FEFD+720637]
	Ordinal0 [0x004A0F3F+724799]
	GetHandleVerifier [0x0089EED2+2769538]
	GetHandleVerifier [0x00890D95+2711877]
	GetHandleVerifier [0x0067A03A+521194]
	GetHandleVerifier [0x00678DA0+516432]
	Ordinal0 [0x0058682C+1665068]
	Ordinal0 [0x0058B128+1683752]
	Ordinal0 [0x0058B215+1683989]
	Ordinal0 [0x00596484+1729668]
	BaseThreadInitThunk [0x756700F9+25]
	RtlGetAppContainerNamedObjectPath [0x77157BBE+286]
	RtlGetAppContainerNamedObjectPath [0x77157B8E+238]
